# Чтение Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (SparkSession.builder
         .appName("Read Session")
         .master("spark://spark-master:7077") 
         .config("spark.jars", 
                 "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar,"
                 "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar,"
                 "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar")
         .getOrCreate()
        )

25/10/29 22:12:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Чтение с локального диска `CSV`

In [ ]:
df = spark.read.csv("/shared_data/bmw.csv", 
                    header=True, 
                    inferSchema=True)

In [ ]:
# Чтение всех файлов из папки
# ВАЖНО! Можно читать только одинаковые по структуре файлы

df = spark.read.csv("/shared_data/*.csv", 
                    header=True)

In [ ]:
# Чтение с параметрами
df = (spark.read
      .option("header", "true")
      .option("delimiter", ",")
      .option("inferSchema", "true")
      .option("encoding", "utf-8")
      .option("nullValue", "NULL")
      .csv("/shared_data/bmw.csv"))

In [ ]:
# Использование формата
df = (spark.read
      .format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load("/shared_data/bmw.csv"))

## Чтение с указанием схемы

In [ ]:
# Чтение CSV с указанием схемы
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Определяем схему для CSV файла
schema = StructType([
    StructField("Model",                StringType(),  True),
    StructField("Year",                 IntegerType(), True),
    StructField("Region",               StringType(),  True),
    StructField("Color",                StringType(),  True),
    StructField("Fuel_Type",            StringType(),  True),
    StructField("Transmission",         StringType(),  True),
    StructField("Engine_Size_L",        DoubleType(),  True),
    StructField("Mileage_KM",           IntegerType(), True),
    StructField("Price_USD",            IntegerType(), True),
    StructField("Sales_Volume",         IntegerType(), True),
    StructField("Sales_Classification", StringType(),  True)
])
            
df = spark.read.csv("/shared_data/bmw.csv",
                    schema=schema,
                    header=True)

## Запись в MinIO `CSV`

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import json

(df.write
   .mode("overwrite")
   .option("header", "false")
   .csv("s3a://learn-bucket/draft/bmw_csv"))

# Спарк по умолчанию не пишет заголовки в CSV файлы
# По этому можно сохранить схему прямо в MinIO

schema_json = df.schema.json()
schema_df = spark.createDataFrame([(schema_json,)], ["schema"])

schema_df.write \
    .mode("overwrite") \
    .option("compression", "none") \
    .text("s3a://learn-bucket/draft/bmw_schema")

## Чтение из MinIO `CSV`

In [ ]:
# Читаем схему напрямую из MinIO
schema_rdd = spark.sparkContext.textFile("s3a://learn-bucket/draft/bmw_schema")
schema_json = schema_rdd.collect()[0]  # берем первую строку

# Восстанавливаем схему
restored_schema = StructType.fromJson(json.loads(schema_json))

# Теперь читаем данные с этой схемой
df = (spark.read
           .schema(restored_schema)
           .option("header", "false")
           .csv("s3a://learn-bucket/draft/bmw_csv"))

# Parquet

In [ ]:
df = (spark.read.parquet("/shared_data/yellow_taxi"))

In [ ]:
df = spark.read.parquet("/shared_data/yellow_taxi/data_*.parquet")

In [ ]:
df = spark.read.parquet("/shared_data/yellow_taxi/single_file.parquet")

## Проверка

In [ ]:
print(f"Количество строк: {df.count()}\n")
df.show(1)
df.printSchema()

In [11]:
spark.stop()